In [19]:
import glob
import fiftyone as fo
import pandas as pd 

images_patt = "../data/reduced_images.train"
y_train = pd.read_csv("../data/tabular/y_train_OXxrJt1.csv")
X_train = pd.read_csv("../data/tabular/X_train_J01Z4CN.csv")
y_pred = pd.read_csv("../data/train_submission.csv")
caption_df = pd.read_csv("../data/image_captions/df.csv")

In [25]:
caption_df.iloc[0]["features"]

"['a bathroom with a tub, toilet, and a window.', 'a bedroom with a bed, chair, and pictures on the wall.', 'a living room with a couch and a table', 'a bedroom with a bed, window, and a rug.']"

In [31]:

# Ex: your custom label format
annotations = {
    "/path/to/images/000001.jpg": "dog",
}

# Create samples for your data
samples = []
for i in range(len(X_train)):
    row = X_train.iloc[i]
    price = y_train.iloc[i]["price"]
    pred = y_pred.iloc[i]["price"]
    id_annonce= int(y_train.iloc[i]["id_annonce"])
    filepaths = glob.glob(f"../data/reduced_images/train/ann_{id_annonce}/*.jpg")
    caption = caption_df.iloc[i]["features"]
    caption=caption.replace('"', "'")
    caption_split = caption.split("',")
    # print(caption)
    # print(len(caption_split))
    # print(len(filepaths))
    for j,filepath in enumerate(filepaths) :
        sample = fo.Sample(filepath=filepath)

        for key in row.keys():
            sample[key]=row[key]
        sample["price"]=fo.Regression(value=price) 
        sample["y_pred"]= fo.Regression(value=pred) 
        sample["caption"]=caption_split[j]
        samples.append(sample)


In [32]:
samples
for col in ["energy_performance_category", "ghg_category","exposition"]:
    for i in range(len(samples)) :
        samples[i][col]=str(samples[i][col])



In [34]:

# Create dataset
dataset = fo.Dataset("Real-estate-dataset-3")
dataset.add_samples(samples)

 100% |███████████| 190699/190699 [2.0m elapsed, 0s remaining, 1.7K samples/s]       


['645e9ae95d1a83b231e8dc41',
 '645e9ae95d1a83b231e8dc42',
 '645e9ae95d1a83b231e8dc43',
 '645e9ae95d1a83b231e8dc44',
 '645e9ae95d1a83b231e8dc45',
 '645e9ae95d1a83b231e8dc46',
 '645e9ae95d1a83b231e8dc47',
 '645e9ae95d1a83b231e8dc48',
 '645e9ae95d1a83b231e8dc49',
 '645e9ae95d1a83b231e8dc4a',
 '645e9ae95d1a83b231e8dc4b',
 '645e9ae95d1a83b231e8dc4c',
 '645e9ae95d1a83b231e8dc4d',
 '645e9ae95d1a83b231e8dc4e',
 '645e9ae95d1a83b231e8dc4f',
 '645e9ae95d1a83b231e8dc50',
 '645e9ae95d1a83b231e8dc51',
 '645e9ae95d1a83b231e8dc52',
 '645e9ae95d1a83b231e8dc53',
 '645e9ae95d1a83b231e8dc54',
 '645e9ae95d1a83b231e8dc55',
 '645e9ae95d1a83b231e8dc56',
 '645e9ae95d1a83b231e8dc57',
 '645e9ae95d1a83b231e8dc58',
 '645e9ae95d1a83b231e8dc59',
 '645e9ae95d1a83b231e8dc5a',
 '645e9ae95d1a83b231e8dc5b',
 '645e9ae95d1a83b231e8dc5c',
 '645e9ae95d1a83b231e8dc5d',
 '645e9ae95d1a83b231e8dc5e',
 '645e9ae95d1a83b231e8dc5f',
 '645e9ae95d1a83b231e8dc60',
 '645e9ae95d1a83b231e8dc61',
 '645e9ae95d1a83b231e8dc62',
 '645e9ae95d1a

In [35]:
results = dataset.evaluate_regressions(
    pred_field ="y_pred",
    gt_field="price",
    eval_key="eval",
)

# Print some standard regression evaluation metrics
results.print_metrics()

# Plot a scatterplot of the results colored by `weather` and scaled by
# `confidence`
plot = results.plot_results()
plot.show()

# # Launch the App to explore
# session = fo.launch_app(dataset)

# # Show the samples with the smallest regression error
# session.view = dataset.sort_by("eval")

# # Show the samples with the largest regression error
# session.view = dataset.sort_by("eval", reverse=True)